<a href="https://colab.research.google.com/github/ayobamialimi/DataScienceEcosystem/blob/main/HWK5_Building_a_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Select a dataset
Andrej Karpathy used a subset of Shakespeare's works in his nanoGPT. **Pick a different author to train this GPT.** For example, [Project Gutenberg](https://www.gutenberg.org) is a library of over 70,000 free eBooks. To download [The Complete Works of William Shakespeare by William Shakespeare](https://www.gutenberg.org/ebooks/100) in text format, you can download the [Plain Text UTF-8](https://www.gutenberg.org/cache/epub/100/pg100.txt) file.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://www.gutenberg.org/cache/epub/13615/pg13615.txt
# Replace "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt" with the URL for your author
# Save the downloaded file as "input.txt"

--2024-03-20 16:15:11--  https://www.gutenberg.org/cache/epub/13615/pg13615.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1816318 (1.7M) [text/plain]
Saving to: ‘pg13615.txt’

pg13615.txt         100%[===================>]   1.73M  4.74MB/s    in 0.4s    

2024-03-20 16:15:12 (4.74 MB/s) - ‘pg13615.txt’ saved [1816318/1816318]



In [ ]:
# read it in to inspect it
with open('/content/pg13615.txt', 'r', encoding='utf-8') as f:
    text = f.read()

## Get the length of this dataset (in characters)

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1785346


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

﻿The Project Gutenberg eBook of Studies in the Psychology of Sex, Volume 6
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Studies in the Psychology of Sex, Volume 6

Author: Havelock Ellis

Release date: October 8, 2004 [eBook #13615]
                Most recently updated: December 18, 2020

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK STUDIES IN THE PSYCHOLOGY OF SEX, VOLUME 6 ***


E-text prepared by Juliet Sutherland and the Project Gutenberg Online
Distributed Proofreading Team (https://www.pgdp.net)



STUDIES IN THE PSYCHOLOGY OF SEX, VOLUME VI

   Sex in Relati

## Get the vocabulary for this dataset, and its length

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz£½¾Æàáâäæçèéêëïñóôöü—‘’“”•™﻿
111


## Create the encoder (string -> list of integers) and decoder (list of integers -> string)

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[64, 65, 65, 1, 76, 64, 61, 74, 61]
hii there


## Load encoded data into a tensor

In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1785346]) torch.int64
tensor([110,  47,  64,  61,   1,  43,  74,  71,  66,  61,  59,  76,   1,  34,
         77,  76,  61,  70,  58,  61,  74,  63,   1,  61,  29,  71,  71,  67,
          1,  71,  62,   1,  46,  76,  77,  60,  65,  61,  75,   1,  65,  70,
          1,  76,  64,  61,   1,  43,  75,  81,  59,  64,  71,  68,  71,  63,
         81,   1,  71,  62,   1,  46,  61,  80,  11,   1,  49,  71,  68,  77,
         69,  61,   1,  21,   0,   1,   1,   1,   1,   0,  47,  64,  65,  75,
          1,  61,  58,  71,  71,  67,   1,  65,  75,   1,  62,  71,  74,   1,
         76,  64,  61,   1,  77,  75,  61,   1,  71,  62,   1,  57,  70,  81,
         71,  70,  61,   1,  57,  70,  81,  79,  64,  61,  74,  61,   1,  65,
         70,   1,  76,  64,  61,   1,  48,  70,  65,  76,  61,  60,   1,  46,
         76,  57,  76,  61,  75,   1,  57,  70,  60,   0,  69,  71,  75,  76,
          1,  71,  76,  64,  61,  74,   1,  72,  57,  74,  76,  75,   1,  71,
         62,   1,  76,  64,  6

## Split the data into training and validation sets

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([110,  47,  64,  61,   1,  43,  74,  71,  66])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([110]) the target: 47
when input is tensor([110,  47]) the target: 64
when input is tensor([110,  47,  64]) the target: 61
when input is tensor([110,  47,  64,  61]) the target: 1
when input is tensor([110,  47,  64,  61,   1]) the target: 43
when input is tensor([110,  47,  64,  61,   1,  43]) the target: 74
when input is tensor([110,  47,  64,  61,   1,  43,  74]) the target: 71
when input is tensor([110,  47,  64,  61,   1,  43,  74,  71]) the target: 66


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[74, 11,  1, 57,  1, 81, 71, 77],
        [72, 61, 71, 72, 68, 61,  0,  1],
        [72, 71, 70, 75, 65, 58, 65, 68],
        [70, 75, 77, 58, 75, 76, 57, 70]])
targets:
torch.Size([4, 8])
tensor([[11,  1, 57,  1, 81, 71, 77, 70],
        [61, 71, 72, 68, 61,  0,  1,  1],
        [71, 70, 75, 65, 58, 65, 68, 65],
        [75, 77, 58, 75, 76, 57, 70, 76]])
----
when input is [74] the target: 11
when input is [74, 11] the target: 1
when input is [74, 11, 1] the target: 57
when input is [74, 11, 1, 57] the target: 1
when input is [74, 11, 1, 57, 1] the target: 81
when input is [74, 11, 1, 57, 1, 81] the target: 71
when input is [74, 11, 1, 57, 1, 81, 71] the target: 77
when input is [74, 11, 1, 57, 1, 81, 71, 77] the target: 70
when input is [72] the target: 61
when input is [72, 61] the target: 71
when input is [72, 61, 71] the target: 72
when input is [72, 61, 71, 72] the target: 68
when input is [72, 61, 71, 72, 68] the target: 61
when input is [72, 6

In [ ]:
print(xb) # our input to the transformer

tensor([[74, 11,  1, 57,  1, 81, 71, 77],
        [72, 61, 71, 72, 68, 61,  0,  1],
        [72, 71, 70, 75, 65, 58, 65, 68],
        [70, 75, 77, 58, 75, 76, 57, 70]])


# 2. Bigram language model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 111])
tensor(5.4346, grad_fn=<NllLossBackward0>)

™/ñü"‘]npQ.%fw.rü"àT—iH#$3Rkm4O$rI.N! T%á™?p—‘S™1j)#X_0OE,'’X/L‘SqtwpFq(n6ñ5VrG
!/X•1½çL•yc!ñ•.E﻿pQw


## Use the AdamW optimizer

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

## Questions
1. (10 points) How many steps did you train the model for?

I trained the model for 4000 steps. I started with 100 steps then increased it to 1000, and then 2000, 3000 and 4000.
2. (15 points) How does the value of the loss function change with the number of steps? Provide a few examples, such as, after 100 steps the loss was 4.656, and after 1,000 steps the loss was 4.231.

At 100 steps, the loss was 5.2366, at 1000 the loss was 4.0174, this shows a gain of 1.2192, and at 2000 steps, the loss was 2.8645 with a sharp devrease of 1.1529. At 3000 steps, the loss was 2.5213 and was 2.4750 at 4000. There is a shart decrease in loss value at the bigining of the training and later the loss value did not show a significant difference. I started the training again and had 2.54 loss at 100 steps and I have 2.440 loss at 1000 steps. This shows a significant difference from the previous set of results


In [ ]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4402759075164795


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


alins a meco athe, e, sg equrive-RSchicedereshld Fü0)™ün offe culllithltuathie atldis aff m  sof ve theng
by al iars biished t ary, S½(ssed (pondivean thess s blithere
  cithsese
 ostas y  heark at."s Zñhexthoifll ont t akrer hes
 f  alirendegrkimedolem schaly tend  o rilld acls opof  athof pon aro pof tionthe ct osed f tiouearecin d —Af wexull ),  quandrs heperel,
 ig t arer f
 d  be t alverarineathelir plat.
palvat womovisthang
fremuio Pralel atly  Stet
  h te LOTha themodugan. t f Thashe'mara


# Optional: The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



# 3. Encoder language model
To run this section of the notebook, you need a GPU (you can run it on CPU but it will take a much longer). You can run it in [Google Colab](https://colab.research.google.com/), and from the menu select **Runtime/Change runtime type** and select **T4 GPU** for **Hardware accelerator**. Then click **Save**.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# You will have to change the path to the 'input.txt' file (e.g., /content/input.txt)
with open('/content/pg13615.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.215663 M parameters
step 0: train loss 4.8891, val loss 4.8821
step 100: train loss 2.6960, val loss 2.8135
step 200: train loss 2.5591, val loss 2.6816
step 300: train loss 2.4640, val loss 2.5873
step 400: train loss 2.3950, val loss 2.4981
step 500: train loss 2.2936, val loss 2.4249
step 600: train loss 2.2377, val loss 2.3617
step 700: train loss 2.2042, val loss 2.3407
step 800: train loss 2.1375, val loss 2.2788
step 900: train loss 2.0941, val loss 2.2727
step 1000: train loss 2.0602, val loss 2.2144
step 1100: train loss 2.0468, val loss 2.2066
step 1200: train loss 2.0154, val loss 2.1764
step 1300: train loss 1.9858, val loss 2.1680
step 1400: train loss 1.9769, val loss 2.1307
step 1500: train loss 1.9534, val loss 2.1350
step 1600: train loss 1.9198, val loss 2.0972
step 1700: train loss 1.8969, val loss 2.0835
step 1800: train loss 1.8897, val loss 2.0748
step 1900: train loss 1.8671, val loss 2.0432
step 2000: train loss 1.8575, val loss 2.0295
step 2100: train loss 1.

## Questions (25 points each)
1. How many attention heads are used in this encoder?

4 attention heads are used in the encoder because n_head is set to 4
2. How many layers are in each feed-forward network? List these layers.

Each feed-forward network consists of 2 linear layers with ReLU function in between and a dropout layer. The number of layers in each feed-forward network is 3
3. How many layers are in the transformer block? List these layers.

The transformer block is made up of the multi-head attention layer (the heads are grouped onto one layer, so there are four attention heads but still only one layer) the feed-forward network (made up of three layers), a normalization layer and a second normalization layer. Therefore, there are 4 main layers in each transformer block but if the component layers of the feed-forward network are being considered (two linear layers and one dropout layer), there are 6 total layers within each block.

# Extra credit (10 points)
How does the **learning rate** affect the model (in terms of learning time, and accuracy)? Pick a different value for the learning rate and run the code to show this.

Smaller learning rates necessitate more training epochs because of the fewer changes and takes more time to complete. On the other hand, larger learning rates result in faster changes. Larger learning rates frequently result in a suboptimal final set of weights and less accurate result. The code below uses a learning rate lr-6 and the validation and train loss with the larger rate is higher for each step of the process and the output indicates higher level of inaccuracy as it produces non-readable words

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-6
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# You will have to change the path to the 'input.txt' file (e.g., /content/input.txt)
with open('/content/pg13615.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.215663 M parameters
step 0: train loss 4.8891, val loss 4.8821
step 100: train loss 4.8636, val loss 4.8619
step 200: train loss 4.8440, val loss 4.8350
step 300: train loss 4.8177, val loss 4.8173
step 400: train loss 4.7923, val loss 4.7917
step 500: train loss 4.7712, val loss 4.7723
step 600: train loss 4.7492, val loss 4.7482
step 700: train loss 4.7214, val loss 4.7256
step 800: train loss 4.6974, val loss 4.7023
step 900: train loss 4.6744, val loss 4.6814
step 1000: train loss 4.6501, val loss 4.6585
step 1100: train loss 4.6284, val loss 4.6333
step 1200: train loss 4.6064, val loss 4.6131
step 1300: train loss 4.5838, val loss 4.5931
step 1400: train loss 4.5636, val loss 4.5669
step 1500: train loss 4.5341, val loss 4.5467
step 1600: train loss 4.5137, val loss 4.5267
step 1700: train loss 4.4859, val loss 4.5037
step 1800: train loss 4.4657, val loss 4.4820
step 1900: train loss 4.4445, val loss 4.4574
step 2000: train loss 4.4183, val loss 4.4361
step 2100: train loss 4.